# 少量样本 QLoRA Finetune Qwen-7B/14B DEMO for Align Tech

## 运行环境确认

In [1]:
import warnings 
warnings.filterwarnings('ignore')

In [2]:
import transformers 
print(transformers.__version__)

4.34.0


In [3]:
import peft 
print(peft.__version__)

0.5.0


In [4]:
import accelerate 
print(accelerate.__version__)

0.23.0


In [5]:
import torchkeras
print(torchkeras.__version__)

3.9.4


In [6]:
!pip show bitsandbytes


Name: bitsandbytes
Version: 0.41.1
Summary: k-bit optimizers and matrix multiplication routines.
Home-page: https://github.com/TimDettmers/bitsandbytes
Author: Tim Dettmers
Author-email: dettmers@cs.washington.edu
License: MIT
Location: /home/ecs-assist-user/micromamba/envs/qwen/lib/python3.10/site-packages
Requires: 
Required-by: 


## 下载目标模型

In [1]:
from modelscope import AutoModelForCausalLM, AutoTokenizer, snapshot_download
target_model="qwen/Qwen-7B-Chat"
#target_model="qwen/Qwen-14B-Chat"
llm_weight_workspace='/mnt/llm-data/'
model_name_or_path=llm_weight_workspace+target_model
llm_finetune_checkpoint_workspace=llm_weight_workspace+'checkpoints/'+target_model
llm_finetune_savepath=llm_weight_workspace+'finetuned/'+target_model
#model_dir = snapshot_download(target_model, revision='v1.0.4',cache_dir=llm_weight_workspace)

2023-10-19 14:41:28,033 - modelscope - INFO - PyTorch version 2.1.0 Found.
2023-10-19 14:41:28,037 - modelscope - INFO - Loading ast index from /home/ecs-assist-user/.cache/modelscope/ast_indexer
2023-10-19 14:41:28,210 - modelscope - INFO - Loading done! Current index file version is 1.9.2, with md5 fd57df25cf18fc0a972c98cfdffc3f42 and a total number of 941 components indexed


## 〇，预训练模型

In [2]:
import warnings
warnings.filterwarnings('ignore')
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM,AutoConfig, AutoModel, BitsAndBytesConfig
from transformers.generation.utils import GenerationConfig
import torch.nn as nn
from torchkeras.chat import ChatLLM 

class ModelLoader(object):  # 创建Circle类
   def __init__(self, model_name_or_path):  # 约定成俗这里应该使用r，它与self.r中的r同名
       self.model_name_or_path = model_name_or_path
       self.bnb_config=BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.float16,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                llm_int8_threshold=6.0,
                llm_int8_has_fp16_weight=False,
            )
   def load(self,for_training=True):  
    self.tokenizer = AutoTokenizer.from_pretrained(
       self.model_name_or_path, trust_remote_code=True)
    if for_training:
        self.model = AutoModelForCausalLM.from_pretrained(self.model_name_or_path,
                    quantization_config=self.bnb_config, 
                    trust_remote_code=True)
        self.model.config.use_cache = False
    else:
        self.model = AutoModelForCausalLM.from_pretrained(self.model_name_or_path,
                    quantization_config=self.bnb_config,
                    trust_remote_code=True)
        self.model.config.use_cache=True
    
    self.model.generation_config = GenerationConfig.from_pretrained(self.model_name_or_path)
    self.llm = ChatLLM(self.model,self.tokenizer)
    return self.model,self.tokenizer,self.llm
    



In [7]:
model_loader=ModelLoader(model_name_or_path=model_name_or_path)
model,tokenizer,llm=model_loader.load()

register magic %%chat sucessed ...


In [3]:
#llm.chat(messages=llm.build_messages(query='Align产品之间的转换有哪些规定?'))

In [8]:
%%chat
什么是智能力引导功能?

智能力引导功能是一种人工智能技术，它可以帮助用户更好地理解和使用人工智能系统。它通过使用自然语言处理、机器学习和深度学习等技术，帮助用户更好地理解人工智能系统的功能和操作方式，并且可以提供有用的建议和指导，以帮助用户更有效地使用人工智能系统。


## 一，准备数据

将原始数据用LLM转换成问答对

In [3]:
%%chat
将以下用···分割的文本中的所有内容转换成多个问题和回答对，将每一个问题的提问用三种不同的方式表达，一个提问匹配一个回答，结果的格式用以下json输出
[{“question”:[“问题1",“问题2”,“问题3"],“answer”:“问题答案“}]
请确保生成的问题列表能涵盖该txt文档中的所有内容，不要有遗漏，保持文本原始的语言，中文的问答依旧用中文，英文的问答依旧用英文
···
什么是智能力引导功能?
智能力引导功能是引导并提供正畸牙齿移动所需的生物力学力量的功能。
传统附件和优化附件有什么区别?
附件本质上是附着在牙齿表面上的材料，包括先使用粘接剂处理牙面，然后再使用复合树脂粘结。附件与矫治器贴合，为牙齿提供矫治力，提供最佳的正畸牙齿移动。
智能力引导功能包括优化附件，它们包括:
用于提供必要的矫治力系统，以实现更多可预测的牙齿移动。
每颗牙齿均根据其宽度，长轴和整个牙齿轮廓等特征使用先进技术进行虚拟建模。
更精确地定位以提供必要的矫治力，同时消除干扰。
与所有智能力引导功能一样，Treat 软件会根据所需的移动自动放置优化附件。
在查看治疗计划时，医生可以虚拟地放置传统附件。
为什么优化附件的形状与矫治器的附件空泡的形状不同? 为什么我看到附件和矫治器之间存在间隙?
优化附件的形状与相应矫治器(上的优化附件位置的)的形状不同。该软件通过两个考虑因素确定矫治器的形状:
矫治器的主动面与优化附件的主动面契合并产生牙齿移动所需的矫治力，力施加垂直于主动面的力，而主动面之间没有间隙。
非主动面所设计的间隙，避免产生不必要的矫治力。
使用优化控根附件时，功能面的正确方向是什么?
这些优化附件是专为上切牙、上下尖牙和前磨牙在整体移动或直立倾斜时，用来控制根倾斜度而设计的。驱动力来自最靠近中心或阻抗中心的附件，或者来自激活的单个控根附件的 SmartStage。远离阻抗中心的附件(即在两个控根优化附件中离咬合边缘嵴或切缘的附件)或单个优化控附件会提供第二个力来产生反力矩用于倾斜控制。
在放置了优化控根附件的上颌侧切牙，矫治器在切端位置部分不贴合。这是正常的吗?
为了产生预期的矫治力系统，我们在矫治器与牙齿切缘近中或远中的位置预留间隙。该间隙称为矫治器间隙。它位于与附件相对的牙齿面，以避免不必要的接触及产生不需要的矫治力。矫治器间隙仅出现在放置了优化控根附件的上颌侧切牙。
对于前磨牙，为什么我会在某些方案上看到两个优化控根附件而在其他方案上只看到一个?
当牙齿有足够的空间时，两个优化控根附件将放置在牙齿的颊面上。当冠高太短时，一个优化控根附件将被放置在颊面上。在最近的更新中，这些附件应与深覆𬌗附件或多颗牙单位附件的一部分区分开来。
如果压力点和矫治器空间有助于侧切牙的控根移动，为什么中切牙上没有?
中切牙是较大的牙齿，并且通常具有足够的牙齿表面以放置两个控根附件，因此不需要压力点和矫治器间隙。
···

{ "question": [ "什么是智能力引导功能?", "智能力引导功能有什么作用?", "优化附件和传统附件有什么区别?" ], "answer": "智能力引导功能是引导并提供正畸牙齿移动所需的生物力学力量的功能。" }, { "question": [ "优化附件包括哪些?", "优化附件如何提供必要的矫治力?", "优化附件与传统附件有何不同?" ], "answer": "优化附件包括用于提供必要的矫治力系统，以实现更多可预测的牙齿移动。每颗牙齿均根据其宽度，长轴和整个牙齿轮廓等特征使用先进技术进行虚拟建模。更精确地定位以提供必要的矫治力，同时消除干扰。与所有智能力引导功能一样，Treat 软件会根据所需的移动自动放置优化附件。" }, { "question": [ "为什么优化附件的形状与矫治器的附件空泡的形状不同?", "为什么我看到附件和矫治器之间存在间隙?" ], "answer": "优化附件的形状与相应矫治器(上的优化附件位置的)的形状不同。该软件通过两个考虑因素确定矫治器的形状:矫治器的主动面与优化附件的主动面契合并产生牙齿移动所需的矫治力，力施加垂直于主动面的力，而主动面之间没有间隙。非主动面所设计的间隙，避免产生不必要的矫治力。" }, { "question": [ "使用优化控根附件时，功能面的正确方向是什么?" ], "answer": "这些优化附件是专为上切牙、上下尖牙和前磨牙在整体移动或直立倾斜时，用来控制根倾斜度而设计的。驱动力来自最靠近中心或阻抗中心的附件，或者来自激活的单个控根附件的 SmartStage。远离阻抗中心的附件(即在两个控根优化附件中离咬合边缘嵴或切缘的附件)或单个优化控附件会提供第二个力来产生反力矩用于倾斜控制。" }, { "question": [ "在放置了优化控根附件的上颌侧切牙，矫治器在切端位置部分不贴合。这是正常的吗?" ], "answer": "为了产生预期的矫治力系统，我们在矫治器与牙齿切缘近中或远中的位置预留间隙。该间隙称为矫治器间隙。它位于


In [9]:
data=[
  {
    "question": [
      "什么是智能力引导功能?",
      "智能力引导功能的作用是什么?",
      "引入智能力引导功能的目的是什么?"
    ],
    "answer": "智能力引导功能是引导并提供正畸牙齿移动所需的生物力学力量的功能。"
  },
  {
    "question": [
      "传统附件和优化附件有什么区别?",
      "请说明传统附件和优化附件的不同之处",
      "优化附件与传统附件相比有哪些特点?"
    ],
    "answer": "附件本质上是附着在牙齿表面上的材料。优化附件包含用于提供必要矫治力系统的先进技术,对每颗牙齿进行虚拟建模,更精确定位以提供必要的矫治力。与传统附件不同,优化附件由Treat软件根据所需移动自动放置。"
  },
  {
    "question": [
      "为什么优化附件的形状与矫治器的形状不同?",
      "优化附件形状与对应矫治器形状不一致的原因是什么?",
      "优化附件与矫治器形状不同的考虑因素有哪些?"
    ],
    "answer": "优化附件的形状与矫治器形状不同。软件通过两个因素确定矫治器形状:主动面之间无间隙以产生牙齿移动所需矫治力;非主动面设计间隙以避免不必要的矫治力。"
  },
  {
    "question": [
      "使用优化控根附件时功能面的正确方向是?",
      "优化控根附件的功能面应该朝向哪个方向?",
      "优化控根附件的功能面正确方向指的是什么?"
    ],
    "answer": "这些优化附件用于控制根倾斜度。驱动力来自最靠近阻抗中心的附件或单个控根附件的SmartStage。远离阻抗中心的附件提供第二个力以产生反力矩用于倾斜控制。"
  },
  {
    "question": [
      "放置优化控根附件的上颌侧切牙,矫治器在切端部分不贴合是否正常?",
      "对于放置了优化控根附件的上颌侧切牙,矫治器在切端有间隙正常吗?",
      "优化控根附件的上颌侧切牙矫治器切端间隙是预期的现象吗?"
    ],
    "answer": "是的。为产生预期的矫治力系统,我们在矫治器与牙齿切缘近中远中留有称为矫治器间隙的间隙,以避免不必要的接触和矫治力。"
  },
  {
    "question": [
      "为什么前磨牙在某些方案见到两个优化控根附件,而其他只见一个?",
      "前磨牙优化控根附件为什么有的方案是两个,有的方案只有一个?",
      "什么情况下前磨牙会用两个优化控根附件,什么情况下只用一个?"
    ],
    "answer": "当牙齿空间足够时会放两个附件在颊面;当冠高太短时会放一个附件在颊面。"
  }, {
    "question": [
      "中切牙没有压力点和矫治器间隙的原因是什么?",
      "为什么中切牙上没有压力点和矫治器空间?",
      "中切牙没有压力点和矫治器间隙的考虑因素是什么?"
    ],
    "answer": "中切牙是较大的牙齿,并且通常具有足够的牙齿表面以放置两个控根附件,因此不需要压力点和矫治器间隙。"
  }]

print(data)

[{'question': ['什么是智能力引导功能?', '智能力引导功能的作用是什么?', '引入智能力引导功能的目的是什么?'], 'answer': '智能力引导功能是引导并提供正畸牙齿移动所需的生物力学力量的功能。'}, {'question': ['传统附件和优化附件有什么区别?', '请说明传统附件和优化附件的不同之处', '优化附件与传统附件相比有哪些特点?'], 'answer': '附件本质上是附着在牙齿表面上的材料。优化附件包含用于提供必要矫治力系统的先进技术,对每颗牙齿进行虚拟建模,更精确定位以提供必要的矫治力。与传统附件不同,优化附件由Treat软件根据所需移动自动放置。'}, {'question': ['为什么优化附件的形状与矫治器的形状不同?', '优化附件形状与对应矫治器形状不一致的原因是什么?', '优化附件与矫治器形状不同的考虑因素有哪些?'], 'answer': '优化附件的形状与矫治器形状不同。软件通过两个因素确定矫治器形状:主动面之间无间隙以产生牙齿移动所需矫治力;非主动面设计间隙以避免不必要的矫治力。'}, {'question': ['使用优化控根附件时功能面的正确方向是?', '优化控根附件的功能面应该朝向哪个方向?', '优化控根附件的功能面正确方向指的是什么?'], 'answer': '这些优化附件用于控制根倾斜度。驱动力来自最靠近阻抗中心的附件或单个控根附件的SmartStage。远离阻抗中心的附件提供第二个力以产生反力矩用于倾斜控制。'}, {'question': ['放置优化控根附件的上颌侧切牙,矫治器在切端部分不贴合是否正常?', '对于放置了优化控根附件的上颌侧切牙,矫治器在切端有间隙正常吗?', '优化控根附件的上颌侧切牙矫治器切端间隙是预期的现象吗?'], 'answer': '是的。为产生预期的矫治力系统,我们在矫治器与牙齿切缘近中远中留有称为矫治器间隙的间隙,以避免不必要的接触和矫治力。'}, {'question': ['为什么前磨牙在某些方案见到两个优化控根附件,而其他只见一个?', '前磨牙优化控根附件为什么有的方案是两个,有的方案只有一个?', '什么情况下前磨牙会用两个优化控根附件,什么情况下只用一个?'], 'answer': '当牙齿空间足够时会放两个附件在颊面;当冠高太短时

### 2，做数据集

In [11]:
from torch.utils.data import Dataset,DataLoader 
from copy import deepcopy
import random
class MyDataset(Dataset):

    def __init__(self,data
                ):
        self.data = data
        self.size = len(data)
        self.index_list = list(range(self.size))
        
        
    def get_messages(self):
        select = random.choice
        messages,history = [],[]
        for t in self.data:
            history.append((select(t["question"]),t["answer"]))
            
        for prompt,response in history:
            pair = [{"role": "user", "content": prompt},
                {"role": "assistant", "content": response}]
            messages.extend(pair)
        return messages 
        
    def __len__(self):
        return self.size
        
    def get(self,index):
        idx = self.index_list[index]
        messages = self.get_messages()
        return messages

    
    def __getitem__(self,index):
        messages = self.get(index)
        input_ids, labels = llm.build_inputs_labels(messages,multi_rounds=True) #支持多轮
        return {'input_ids':input_ids,'labels':labels}
    

In [12]:
#slider=int(round((len(data)*0.7),0))
#ds_train =  MyDataset(data[:slider])
#ds_val = MyDataset(data[slider:])
ds_train=ds_val= MyDataset(data)

In [13]:
ds_val

In [14]:
ds_train.get_messages()

[{'role': 'user', 'content': '智能力引导功能的作用是什么?'},
 {'role': 'assistant', 'content': '智能力引导功能是引导并提供正畸牙齿移动所需的生物力学力量的功能。'},
 {'role': 'user', 'content': '请说明传统附件和优化附件的不同之处'},
 {'role': 'assistant',
  'content': '附件本质上是附着在牙齿表面上的材料。优化附件包含用于提供必要矫治力系统的先进技术,对每颗牙齿进行虚拟建模,更精确定位以提供必要的矫治力。与传统附件不同,优化附件由Treat软件根据所需移动自动放置。'},
 {'role': 'user', 'content': '为什么优化附件的形状与矫治器的形状不同?'},
 {'role': 'assistant',
  'content': '优化附件的形状与矫治器形状不同。软件通过两个因素确定矫治器形状:主动面之间无间隙以产生牙齿移动所需矫治力;非主动面设计间隙以避免不必要的矫治力。'},
 {'role': 'user', 'content': '优化控根附件的功能面应该朝向哪个方向?'},
 {'role': 'assistant',
  'content': '这些优化附件用于控制根倾斜度。驱动力来自最靠近阻抗中心的附件或单个控根附件的SmartStage。远离阻抗中心的附件提供第二个力以产生反力矩用于倾斜控制。'},
 {'role': 'user', 'content': '优化控根附件的上颌侧切牙矫治器切端间隙是预期的现象吗?'},
 {'role': 'assistant',
  'content': '是的。为产生预期的矫治力系统,我们在矫治器与牙齿切缘近中远中留有称为矫治器间隙的间隙,以避免不必要的接触和矫治力。'},
 {'role': 'user', 'content': '前磨牙优化控根附件为什么有的方案是两个,有的方案只有一个?'},
 {'role': 'assistant', 'content': '当牙齿空间足够时会放两个附件在颊面;当冠高太短时会放一个附件在颊面。'},
 {'role': 'user', 'content': '为什么中切牙上没有压力点和矫治器空间?'},
 {'r

### 3，创建管道

In [15]:
#如果pad为None，需要处理一下
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.unk_token_id if tokenizer.unk_token_id is not None else tokenizer.eos_token_id
    

def data_collator(examples: list):
    
    len_ids = [len(example["input_ids"]) for example in examples]
    longest = max(len_ids) #之后按照batch中最长的input_ids进行padding
    
    input_ids = []
    labels_list = []
    
    for length, example in sorted(zip(len_ids, examples), key=lambda x: -x[0]):
        ids = example["input_ids"]
        labs = example["labels"]
        
        ids = ids + [tokenizer.pad_token_id] * (longest - length)
        labs = labs + [-100] * (longest - length)
        
        input_ids.append(torch.LongTensor(ids))
        labels_list.append(torch.LongTensor(labs))
          
    input_ids = torch.stack(input_ids)
    labels = torch.stack(labels_list)
    return {
        "input_ids": input_ids,
        "labels": labels,
    }


In [16]:
import torch 
dl_train = torch.utils.data.DataLoader(ds_train,batch_size=1,
                                       pin_memory=False,shuffle=False,
                                       collate_fn = data_collator)

dl_val = torch.utils.data.DataLoader(ds_val,batch_size=1,
                                    pin_memory=False,shuffle=False,
                                     collate_fn = data_collator)


In [17]:
for batch in dl_train:
    pass

In [18]:
#试跑一个batch
#out = model(**batch)

In [13]:
#out.loss 

In [14]:
#len(dl_train)

## 二，定义模型

下面我们将使用QLoRA(实际上用的是量化的AdaLoRA）算法来微调Baichuan-13b模型。

In [15]:
from peft import get_peft_config, get_peft_model, TaskType
model.supports_gradient_checkpointing = True  #
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!


In [16]:
import bitsandbytes as bnb 
def find_all_linear_names(model):
    """
    找出所有全连接层，为所有全连接添加adapter
    """
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:  # needed for 16-bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)


In [17]:
from peft import prepare_model_for_kbit_training 
model = prepare_model_for_kbit_training(model)


In [18]:
lora_modules = find_all_linear_names(model)
print(lora_modules) 


['c_attn', 'w2', 'c_proj', 'w1']


In [19]:
from peft import AdaLoraConfig
peft_config = AdaLoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=False,
    r=16,
    lora_alpha=16, lora_dropout=0.08,
    target_modules= lora_modules
)

peft_model = get_peft_model(model, peft_config)

peft_model.is_parallelizable = True
peft_model.model_parallel = True
peft_model.print_trainable_parameters()


trainable params: 41,843,040 || all params: 14,209,134,120 || trainable%: 0.2944798722189836


## 三，训练模型

In [20]:
from torchkeras import KerasModel 
from accelerate import Accelerator 

class StepRunner:
    def __init__(self, net, loss_fn, accelerator=None, stage = "train", metrics_dict = None, 
                 optimizer = None, lr_scheduler = None
                 ):
        self.net,self.loss_fn,self.metrics_dict,self.stage = net,loss_fn,metrics_dict,stage
        self.optimizer,self.lr_scheduler = optimizer,lr_scheduler
        self.accelerator = accelerator if accelerator is not None else Accelerator() 
        torch.cuda.empty_cache()
        if self.stage=='train':
            self.net.train() 
        else:
            self.net.eval()
    
    def __call__(self, batch):
        
        #loss
        with self.accelerator.autocast():
            loss = self.net.forward(**batch)[0]
            

        #backward()
        if self.optimizer is not None and self.stage=="train":
            torch.cuda.empty_cache()
            self.accelerator.backward(loss)
            torch.cuda.empty_cache()
            if self.accelerator.sync_gradients:
                self.accelerator.clip_grad_norm_(self.net.parameters(), 1.0)
            self.optimizer.step()
            if self.lr_scheduler is not None:
                self.lr_scheduler.step()
            self.optimizer.zero_grad()
            
        all_loss = self.accelerator.gather(loss).sum()
        
        #losses (or plain metrics that can be averaged)
        step_losses = {self.stage+"_loss":all_loss.item()}
        
        #metrics (stateful metrics)
        step_metrics = {}
        
        if self.stage=="train":
            if self.optimizer is not None:
                step_metrics['lr'] = self.optimizer.state_dict()['param_groups'][0]['lr']
            else:
                step_metrics['lr'] = 0.0
        return step_losses,step_metrics
    
KerasModel.StepRunner = StepRunner 

#仅仅保存QLora可训练参数
def save_ckpt(self, ckpt_path='checkpoint', accelerator = None):
    unwrap_net = accelerator.unwrap_model(self.net)
    unwrap_net.save_pretrained(ckpt_path)
    
def load_ckpt(self, ckpt_path='checkpoint'):
    import os
    self.net.load_state_dict(
        torch.load(os.path.join(ckpt_path,'adapter_model.bin')),strict =False)
    self.from_scratch = False
    
KerasModel.save_ckpt = save_ckpt 
KerasModel.load_ckpt = load_ckpt 


In [21]:
optimizer = bnb.optim.adamw.AdamW(peft_model.parameters(),
                                  lr=6e-03,is_paged=True)  #'paged_adamw'
keras_model = KerasModel(peft_model,loss_fn =None,
        optimizer=optimizer,accelerator=Accelerator(mixed_precision="no",cpu=False,
            gradient_accumulation_steps=1,num_processes=4)) 

ckpt_path = llm_finetune_checkpoint_workspace

TypeError: Accelerator.__init__() got an unexpected keyword argument 'num_processes'

In [ ]:
keras_model.from_scratch=False
torch.cuda.empty_cache()

In [ ]:
# keras_model.load_ckpt(ckpt_path) #支持加载微调后的权重继续训练(断点续训)
keras_model.fit(train_data = dl_train,
                val_data = dl_val, 
                epochs=100,patience=15,
                monitor='val_loss',mode='min',
                ckpt_path = ckpt_path
               )

## 四，保存模型

为减少GPU压力，此处可重启kernel释放显存

In [31]:
import warnings 
warnings.filterwarnings('ignore')

In [32]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM,AutoConfig, AutoModel, BitsAndBytesConfig
from transformers.generation.utils import GenerationConfig
import torch.nn as nn
#使用QLoRA引入的 NF4量化数据类型以节约显存

ckpt_path = ckpt_path



tokenizer = AutoTokenizer.from_pretrained(
   model_name_or_path, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                trust_remote_code=True) 

model.generation_config = GenerationConfig.from_pretrained(model_name_or_path)


The model is automatically converting to fp16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [33]:
from peft import PeftModel

#可能需要5分钟左右
peft_model = PeftModel.from_pretrained(model, ckpt_path)
model_new = peft_model.merge_and_unload()


In [34]:
from transformers.generation.utils import GenerationConfig
model_new.generation_config = GenerationConfig.from_pretrained(model_name_or_path)


In [35]:
save_path = llm_finetune_savepath

In [36]:
tokenizer.save_pretrained(save_path)
model_new.save_pretrained(save_path)

In [37]:
import os
a = os.system("cp "+model_name_or_path+"/*.py "+save_path)
print(a)

0


In [38]:
import os
a = os.system("ls "+save_path)

config.json
configuration_qwen.py
cpp_kernels.py
generation_config.json
modeling_qwen.py
pytorch_model-00001-of-00002.bin
pytorch_model-00002-of-00002.bin
pytorch_model.bin.index.json
qwen_generation_utils.py
qwen.tiktoken
special_tokens_map.json
tokenization_qwen.py
tokenizer_config.json


## 五，使用模型

为减少GPU压力，此处可再次重启kernel释放显存。

In [3]:

import warnings
warnings.filterwarnings('ignore')


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM,AutoConfig, AutoModel, BitsAndBytesConfig
from transformers.generation.utils import GenerationConfig
import torch.nn as nn
model_loader=ModelLoader(model_name_or_path=llm_finetune_savepath)
model_loader.load(for_training=False)

register magic %%chat sucessed ...
Using bos_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


(QWenLMHeadModel(
   (transformer): QWenModel(
     (wte): Embedding(151936, 4096)
     (drop): Dropout(p=0.0, inplace=False)
     (rotary_emb): RotaryEmbedding()
     (h): ModuleList(
       (0-31): 32 x QWenBlock(
         (ln_1): RMSNorm()
         (attn): QWenAttention(
           (c_attn): Linear4bit(in_features=4096, out_features=12288, bias=True)
           (c_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
           (attn_dropout): Dropout(p=0.0, inplace=False)
         )
         (ln_2): RMSNorm()
         (mlp): QWenMLP(
           (w1): Linear4bit(in_features=4096, out_features=11008, bias=False)
           (w2): Linear4bit(in_features=4096, out_features=11008, bias=False)
           (c_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
         )
       )
     )
     (ln_f): RMSNorm()
   )
   (lm_head): Linear(in_features=4096, out_features=151936, bias=False)
 ),
 QWenTokenizer(name_or_path='/mnt/llm-data/finetuned/qwen/Qwen-7B-Chat', v

我们测试一下微调后的效果。

先用训练集中的一条数据中的三种不同的问法提问，原始训练数据如下：
```json
{
    "question": [
      "什么是智能力引导功能?",
      "智能力引导功能的作用是什么?",
      "引入智能力引导功能的目的是什么?"
    ],
    "answer": "智能力引导功能是引导并提供正畸牙齿移动所需的生物力学力量的功能。"
  }
```

In [4]:
%%chat
什么是智能力引导功能?

智能力引导功能是引导并提供正畸牙齿移动所需的生物力学力量的功能。


In [5]:
%%chat
智能力引导功能的作用是什么?

智能力引导功能是引导并提供正畸牙齿移动所需的生物力学力量的功能。


In [43]:
%%chat
引入智能力引导功能的目的是什么?

智能力引导功能是隐形式力,用于更精细地控制牙齿移动,因此能提供超凡的矫治力控制。


同时问两个问题

```json
原始训练数据
{
    "question": [
      "为什么优化附件的形状与矫治器的形状不同?",
      "优化附件形状与对应矫治器形状不一致的原因是什么?",
      "优化附件与矫治器形状不同的考虑因素有哪些?"
    ],
    "answer": "优化附件的形状与矫治器形状不同。软件通过两个因素确定矫治器形状:主动面之间无间隙以产生牙齿移动所需矫治力;非主动面设计间隙以避免不必要的矫治力。"
  },
  {
    "question": [
      "为什么前磨牙在某些方案见到两个优化控根附件,而其他只见一个?",
      "前磨牙优化控根附件为什么有的方案是两个,有的方案只有一个?",
      "什么情况下前磨牙会用两个优化控根附件,什么情况下只用一个?"
    ],
    "answer": "当牙齿空间足够时会放两个附件在颊面;当冠高太短时会放一个附件在颊面。"
  }
  ```

In [5]:
%%chat
请回答两个问题，1.为什么优化附件的形状与矫治器的形状不同? 2.为什么前磨牙在某些方案见到两个优化控根附件,而其他只见一个?

1. 优化附件形状与矫治器形状不同,因为前者需要考虑根倾斜、颊面倾斜以及可能的牙齿冠高。2. 前磨牙牙齿移动速度比后牙快,因此在某些正畸方案中需要使用两个优化附件以增加控制力。


In [6]:
%%chat
刚才问了什么问题？

您刚才问了两个问题。第一个问题是：“为什么优化附件的形状与矫治器的形状不同?” 第二个问题是：“为什么前磨牙在某些方案见到两个优化控根附件,而其他只见一个?”。


```json
{
    "question": [
      "放置优化控根附件的上颌侧切牙,矫治器在切端部分不贴合是否正常?",
      "对于放置了优化控根附件的上颌侧切牙,矫治器在切端有间隙正常吗?",
      "优化控根附件的上颌侧切牙矫治器切端间隙是预期的现象吗?"
    ],
    "answer": "是的。为产生预期的矫治力系统,我们在矫治器与牙齿切缘近中远中留有称为矫治器间隙的间隙,以避免不必要的接触和矫治力。"
  }
```

In [49]:
%%chat
对于放置了优化控根附件的上颌侧切牙,矫治器在切端有间隙正常吗?

是的。为产生预期的矫治力系统,我们在矫治器与牙齿切缘近中远中留有称为矫治器间隙的间隙,以避免不必要的接触和矫治力。


In [ ]:
%%chat
刚才问了什么问题？

In [45]:
%%chat
刚才问了几个问题？

是的，您刚才问了几个问题。我是一个大型语言模型，可以回答各种问题。如果您有任何问题，请随时告诉我。我会尽力提供帮助。
